In [1]:
import os
from pathlib import Path
import sys
import pandas as pd
import numpy as np
notebook_path = os.getcwd()
ROOT_dir = Path(notebook_path).parent.absolute()
sys.path.append(str(ROOT_dir))
from src.textSummarizer.logging.logging import logger
print(f"Current directory: {os.getcwd()}")
os.chdir("..")
print(f"Current directory: {os.getcwd()}")

Current directory: d:\1Acads_local\AI_related_projects\github_projects\text_summeriser_project\Text_summerizer_website_intiail\research
Current directory: d:\1Acads_local\AI_related_projects\github_projects\text_summeriser_project\Text_summerizer_website_intiail


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int

In [3]:
from src.textSummarizer.constants import config_yaml_file_path,prams_yaml_file_path
from src.textSummarizer.utils.common import read_yaml,create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = config_yaml_file_path,
        params_filepath = prams_yaml_file_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
        )

        return model_trainer_config

D:\1Acads_local\AI_related_projects\github_projects\text_summeriser_project\Text_summerizer_website_intiail\config\config.yaml
D:\1Acads_local\AI_related_projects\github_projects\text_summeriser_project\Text_summerizer_website_intiail\params.yaml


In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
import os


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_from_disk
from huggingface_hub import login
import os
from src.textSummarizer.logging.logging import logger

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
        # Initialize HF authentication if token exists
        if hasattr(self.config, 'hf_token') and self.config.hf_token:
            logger.info("Authenticating with Hugging Face hub")
            login(token=self.config.hf_token)
            self.use_auth = True
        else:
            logger.warning("No HF token provided - attempting without authentication")
            self.use_auth = False

    def train(self):
        try:
            logger.info("Initializing training components")
            device = "cpu"
            
            # Initialize tokenizer with auth
            logger.info("Loading tokenizer")
            tokenizer = AutoTokenizer.from_pretrained(
                self.config.model_ckpt,
                use_auth_token=self.config.hf_token if self.use_auth else None
            )
            
            # Initialize model with auth
            logger.info("Loading model")
            model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
                self.config.model_ckpt,
                use_auth_token=self.config.hf_token if self.use_auth else None
            ).to(device)
            
            # Initialize data collator
            logger.info("Setting up data collator")
            seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
            
            # Load dataset
            # Fix path resolution using Path and os.path
            data_set_location = self.config.data_path  # artifacts/data_transformation/samsum_dataset
            #ROOT_dir = Path(notebook_path).parent.absolute()  # Fix: removed extra parentheses
            #data_set_location = os.path.join(ROOT_dir, data_set_location)  # Join paths correctly

            # Ensure path exists
            if not os.path.exists(data_set_location):
                raise FileNotFoundError(f"Dataset not found at: {data_set_location}")
            else:
                logger.info(f"Loading dataset from {data_set_location} and path = {os.getcwd()}")

            logger.info(f"Loading dataset from {data_set_location}")
            dataset_samsum_pt = load_from_disk(data_set_location)
            logger.info(f">>>>>>>>>>>>>>>>>> data loading successful <<<<<<<<<<<<<<<<<<<<")
            
            # Training arguments
            logger.info("Configuring training arguments")
            training_args = TrainingArguments(
                output_dir=self.config.root_dir,
                num_train_epochs=self.config.num_train_epochs,
                warmup_steps=self.config.warmup_steps,
                per_device_train_batch_size=self.config.per_device_train_batch_size,
                weight_decay=self.config.weight_decay,
                logging_steps=self.config.logging_steps,
            )

            # Initialize Trainer
            logger.info("Initializing Trainer")
            trainer = Trainer(
                model=model_pegasus,
                args=training_args,
                train_dataset=dataset_samsum_pt['test'],
                eval_dataset=dataset_samsum_pt['validation'],
                data_collator=seq2seq_data_collator,
                tokenizer=tokenizer
            )

            # Start training
            logger.info("Starting training")
            trainer.train()

        except Exception as e:
            logger.info(f"An error occurred during training: {e}")
            raise e

        

c:\Users\gurra\anaconda3\envs\myenvtext\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-26 17:28:12,524: INFO: config: PyTorch version 2.4.1 available.]


In [5]:
from src.textSummarizer.config.configuration import  ConfigurationManager


class ModelTrainerTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        try:
            config = ConfigurationManager()
            model_trainer_config = config.get_model_trainer_config() #This will return all the weights and ckpt file and path of dataset etc....
            model_trainer = ModelTrainer(model_trainer_config)
            model_trainer.train()
        except Exception as e:
            print(e)



In [6]:
import psutil
import time
import sys
import threading
from src.textSummarizer.logging.logging import logger

def monitor_cpu_usage(threshold=50, check_interval=5):
    """
    Monitor CPU usage and terminate the process if it exceeds the threshold.
    :param threshold: CPU usage percentage threshold to trigger termination.
    :param check_interval: Time interval (in seconds) between CPU usage checks.
    """
    while True:
        cpu_usage = psutil.cpu_percent(interval=1)
        if cpu_usage > threshold:
            print(f"CPU usage exceeded {threshold}%. Terminating training...")
            sys.exit(1)
        time.sleep(check_interval)

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    
    try:
        print("Starting model training...")
        
        # Start CPU monitoring in a separate thread
        monitor_thread = threading.Thread(target=monitor_cpu_usage, args=(50, 5))
        monitor_thread.daemon = True
        monitor_thread.start()
        
        model_trainer.train()
        print("Training completed successfully")
        
    except KeyboardInterrupt:
        print("\nTraining interrupted by user. Saving checkpoint...")
        # Save checkpoint logic here if needed
        model_trainer.save_checkpoint()
        print("Checkpoint saved. Exiting gracefully.")
        sys.exit(0)
        
    except Exception as training_error:
        print(f"Training error occurred: {str(training_error)}")
        # Log the error details
        logger.info(f"Training failed with error: {str(training_error)}")
        raise

except Exception as setup_error:
    print(f"Setup error occurred: {str(setup_error)}")
    logger.info(f"Setup failed with error: {str(setup_error)}")
    raise

[2024-12-26 17:28:16,748: INFO: common: yaml file: D:\1Acads_local\AI_related_projects\github_projects\text_summeriser_project\Text_summerizer_website_intiail\config\config.yaml loaded successfully]
[2024-12-26 17:28:16,754: INFO: common: yaml file: D:\1Acads_local\AI_related_projects\github_projects\text_summeriser_project\Text_summerizer_website_intiail\params.yaml loaded successfully]
[2024-12-26 17:28:16,756: INFO: common: created directory at: artifacts]
[2024-12-26 17:28:16,759: INFO: common: created directory at: artifacts/model_trainer]
[2024-12-26 17:28:16,761: WARNING: 715066722: No HF token provided - attempting without authentication]
Starting model training...
[2024-12-26 17:28:16,767: INFO: 715066722: Initializing training components]
[2024-12-26 17:28:16,768: INFO: 715066722: Loading tokenizer]
[2024-12-26 17:28:19,525: INFO: 715066722: Loading model]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2024-12-26 17:28:27,850: INFO: 715066722: Setting up data collator]
[2024-12-26 17:28:27,852: INFO: 715066722: Loading dataset from artifacts/data_transformation/samsum_dataset and path = d:\1Acads_local\AI_related_projects\github_projects\text_summeriser_project\Text_summerizer_website_intiail]
[2024-12-26 17:28:27,853: INFO: 715066722: Loading dataset from artifacts/data_transformation/samsum_dataset]
[2024-12-26 17:28:27,918: INFO: 715066722: >>>>>>>>>>>>>>>>>> data loading successful <<<<<<<<<<<<<<<<<<<<]
[2024-12-26 17:28:27,919: INFO: 715066722: Configuring training arguments]
[2024-12-26 17:28:27,932: INFO: 715066722: Initializing Trainer]


C:\Users\gurra\AppData\Local\Temp\ipykernel_33924\715066722.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[2024-12-26 17:28:28,332: INFO: 715066722: Starting training]


  0%|          | 4/819 [00:40<2:01:34,  8.95s/it]

CPU usage exceeded 50%. Terminating training...

Training interrupted by user. Saving checkpoint...
Setup error occurred: 'ModelTrainer' object has no attribute 'save_checkpoint'
[2024-12-26 17:29:31,130: INFO: 3154678626: Setup failed with error: 'ModelTrainer' object has no attribute 'save_checkpoint']


AttributeError: 'ModelTrainer' object has no attribute 'save_checkpoint'